### Libraries

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# (Optional) If you're working inside Jupyter
%load_ext autoreload
%autoreload 2
%matplotlib inline



In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
from data_processor_v2 import DataProcessorUpdated

processor = DataProcessorUpdated()

In [ ]:
processor.load_and_clean_data()

In [ ]:
processor.combine_all_data()

In [ ]:
processor.df

In [ ]:
processor.shift_dap()

In [ ]:
processor.df

In [ ]:
processor.split_data(feature_columns=['DAP_SystemLambda', 'SCED_system_lambda'])

In [ ]:
processor.standardize_data()

In [ ]:
processor.shift_data()

In [ ]:
x_train_mlp, x_val_mlp, x_test_mlp, y_train_mlp, y_val_mlp, y_test_mlp = processor.flatten()

In [ ]:
len(x_train_mlp[1])

# Model Building

In [ ]:
from MLP import MLPModel

new_model = MLPModel(
    input_shape=len(x_train_mlp[1]),    # 5 features
    output_shape=len(y_train_mlp[1]),   # 1 price per time step
    hidden_layers=[512, 512, 512]  # or [256, 256]
)

mlp_model = new_model.get_model()
mlp_model.to(device)


In [ ]:
from ModelTrainer import ModelTrainer
from losses import fluctuation_loss  # or define it above in your notebook

trainer = ModelTrainer(
    model=mlp_model,
    features_training_data=x_train_mlp,
    target_training_data=y_train_mlp,
    features_eval_data=x_val_mlp,
    target_eval_data=y_val_mlp,
    device=device,
    loss_fn=lambda pred, target: fluctuation_loss(pred, target, alpha=0.2)
)


trainer.train(epochs=100, batch_size=32, patience=50, learning_rate=5e-4)


In [19]:
import optuna
from MLP import MLPModel
from ModelTrainer import ModelTrainer
from losses import fluctuation_loss

def objective(trial):
    # Suggest hyperparameters
    hidden_dim = trial.suggest_categorical("hidden_dim", [128, 256, 512])
    n_layers = trial.suggest_int("n_layers", 2, 4)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    alpha = trial.suggest_uniform("alpha", 0.1, 0.5)

    # Build hidden layers
    hidden_layers = [hidden_dim] * n_layers

    # Build model
    model = MLPModel(input_shape=len(x_train_mlp[1]), output_shape=len(y_train_mlp[1]), hidden_layers=hidden_layers).get_model().to(device)

    # Trainer
    trainer = ModelTrainer(
        model=model,
        features_training_data=x_train_mlp,
        target_training_data=y_train_mlp,
        features_eval_data=x_val_mlp,
        target_eval_data=y_val_mlp,
        device=device,
        loss_fn=lambda pred, target: fluctuation_loss(pred, target, alpha=alpha)
    )

    # Train for a few epochs
    trainer.train(epochs=50, batch_size=32, patience=10, learning_rate=lr)

    # Return final eval loss (or early stopping best)
    return trainer.history['eval_loss'][-1]


In [ ]:

study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.HyperbandPruner()
)
study.optimize(objective, n_trials=50)


[I 2025-05-10 02:05:33,937] A new study created in memory with name: no-name-d0b6b932-da01-495a-bb41-76258bea75d2
/ephnvme/weiliang/cache/tmpdir/ipykernel_2964779/1319189539.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
/ephnvme/weiliang/cache/tmpdir/ipykernel_2964779/1319189539.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.1, 0.5)


Epoch 1: Train Loss = 0.7148, Eval Loss = 0.2653
Epoch 2: Train Loss = 0.5694, Eval Loss = 0.2650
Epoch 3: Train Loss = 0.5189, Eval Loss = 0.2620
Epoch 4: Train Loss = 0.4657, Eval Loss = 0.2610
Epoch 5: Train Loss = 0.4194, Eval Loss = 0.2598
Epoch 6: Train Loss = 0.3814, Eval Loss = 0.2611
Epoch 7: Train Loss = 0.3520, Eval Loss = 0.2601
Epoch 8: Train Loss = 0.3284, Eval Loss = 0.2615
Epoch 9: Train Loss = 0.3198, Eval Loss = 0.2617
Epoch 10: Train Loss = 0.3108, Eval Loss = 0.2601
Epoch 11: Train Loss = 0.2972, Eval Loss = 0.2613
Epoch 12: Train Loss = 0.2825, Eval Loss = 0.2611
Epoch 13: Train Loss = 0.2743, Eval Loss = 0.2620
Epoch 14: Train Loss = 0.2658, Eval Loss = 0.2624


[I 2025-05-10 02:06:12,033] Trial 0 finished with value: 0.2641529440879822 and parameters: {'hidden_dim': 512, 'n_layers': 2, 'lr': 0.0001306215766594925, 'alpha': 0.26148110433114424}. Best is trial 0 with value: 0.2641529440879822.


Epoch 15: Train Loss = 0.2619, Eval Loss = 0.2642
Early stopping triggered at epoch 15
Epoch 1: Train Loss = 0.7178, Eval Loss = 0.2587
Epoch 2: Train Loss = 0.5661, Eval Loss = 0.2565
Epoch 3: Train Loss = 0.5481, Eval Loss = 0.2549
Epoch 4: Train Loss = 0.5182, Eval Loss = 0.2535
Epoch 5: Train Loss = 0.5060, Eval Loss = 0.2518
Epoch 6: Train Loss = 0.4756, Eval Loss = 0.2492
Epoch 7: Train Loss = 0.4433, Eval Loss = 0.2476
Epoch 8: Train Loss = 0.4021, Eval Loss = 0.2464
Epoch 9: Train Loss = 0.3845, Eval Loss = 0.2480
Epoch 10: Train Loss = 0.3428, Eval Loss = 0.2468
Epoch 11: Train Loss = 0.3244, Eval Loss = 0.2471
Epoch 12: Train Loss = 0.3040, Eval Loss = 0.2467
Epoch 13: Train Loss = 0.2926, Eval Loss = 0.2487
Epoch 14: Train Loss = 0.2926, Eval Loss = 0.2501
Epoch 15: Train Loss = 0.2904, Eval Loss = 0.2506
Epoch 16: Train Loss = 0.2743, Eval Loss = 0.2509
Epoch 17: Train Loss = 0.2727, Eval Loss = 0.2511


[I 2025-05-10 02:07:08,336] Trial 1 finished with value: 0.2538983225822449 and parameters: {'hidden_dim': 256, 'n_layers': 4, 'lr': 0.0001356084390443327, 'alpha': 0.18976033190493774}. Best is trial 1 with value: 0.2538983225822449.


...

Epoch 13: Train Loss = 0.5010, Eval Loss = 0.2503
Early stopping triggered at epoch 13
Epoch 1: Train Loss = 0.7793, Eval Loss = 0.2436
Epoch 2: Train Loss = 0.5751, Eval Loss = 0.2439
Epoch 3: Train Loss = 0.5160, Eval Loss = 0.2438
Epoch 4: Train Loss = 0.5033, Eval Loss = 0.2425
Epoch 5: Train Loss = 0.4744, Eval Loss = 0.2409
Epoch 6: Train Loss = 0.4575, Eval Loss = 0.2407
Epoch 7: Train Loss = 0.4404, Eval Loss = 0.2388
Epoch 8: Train Loss = 0.4232, Eval Loss = 0.2381
Epoch 9: Train Loss = 0.4067, Eval Loss = 0.2373
Epoch 10: Train Loss = 0.3949, Eval Loss = 0.2369
Epoch 11: Train Loss = 0.3768, Eval Loss = 0.2373
Epoch 12: Train Loss = 0.3639, Eval Loss = 0.2371
Epoch 13: Train Loss = 0.3509, Eval Loss = 0.2371
Epoch 14: Train Loss = 0.3402, Eval Loss = 0.2366
Epoch 15: Train Loss = 0.3295, Eval Loss = 0.2372
Epoch 16: Train Loss = 0.3195, Eval Loss = 0.2372
Epoch 17: Train Loss = 0.3112, Eval Loss = 0.2371
Epoch 18: Train Loss = 0.3018, Eval Loss = 0.2373
Epoch 19: Train Loss =

[I 2025-05-10 02:45:41,668] Trial 41 finished with value: 0.23695778846740723 and parameters: {'hidden_dim': 256, 'n_layers': 2, 'lr': 4.367489883480501e-05, 'alpha': 0.1060081383810343}. Best is trial 26 with value: 0.23519575595855713.


Epoch 24: Train Loss = 0.2613, Eval Loss = 0.2370
Early stopping triggered at epoch 24
Epoch 1: Train Loss = 0.8463, Eval Loss = 0.2465
Epoch 2: Train Loss = 0.6061, Eval Loss = 0.2475
Epoch 3: Train Loss = 0.5437, Eval Loss = 0.2461
Epoch 4: Train Loss = 0.5181, Eval Loss = 0.2457
Epoch 5: Train Loss = 0.4957, Eval Loss = 0.2448
Epoch 6: Train Loss = 0.4822, Eval Loss = 0.2440
Epoch 7: Train Loss = 0.4661, Eval Loss = 0.2428
Epoch 8: Train Loss = 0.4529, Eval Loss = 0.2421
Epoch 9: Train Loss = 0.4380, Eval Loss = 0.2413
Epoch 10: Train Loss = 0.4253, Eval Loss = 0.2412
Epoch 11: Train Loss = 0.4134, Eval Loss = 0.2409
Epoch 12: Train Loss = 0.3987, Eval Loss = 0.2410
Epoch 13: Train Loss = 0.3898, Eval Loss = 0.2403
Epoch 14: Train Loss = 0.3794, Eval Loss = 0.2403
Epoch 15: Train Loss = 0.3685, Eval Loss = 0.2404
Epoch 16: Train Loss = 0.3592, Eval Loss = 0.2402
Epoch 17: Train Loss = 0.3489, Eval Loss = 0.2402
Epoch 18: Train Loss = 0.3409, Eval Loss = 0.2403
Epoch 19: Train Loss =

[I 2025-05-10 02:46:39,703] Trial 42 finished with value: 0.24114267528057098 and parameters: {'hidden_dim': 256, 'n_layers': 2, 'lr': 3.432532386007783e-05, 'alpha': 0.1235023036594349}. Best is trial 26 with value: 0.23519575595855713.


Epoch 31: Train Loss = 0.2619, Eval Loss = 0.2411
Early stopping triggered at epoch 31
Epoch 1: Train Loss = 0.7421, Eval Loss = 0.2429
Epoch 2: Train Loss = 0.5315, Eval Loss = 0.2426
Epoch 3: Train Loss = 0.4928, Eval Loss = 0.2416
Epoch 4: Train Loss = 0.4644, Eval Loss = 0.2403
Epoch 5: Train Loss = 0.4415, Eval Loss = 0.2381
Epoch 6: Train Loss = 0.4175, Eval Loss = 0.2378
Epoch 7: Train Loss = 0.3945, Eval Loss = 0.2366
Epoch 8: Train Loss = 0.3718, Eval Loss = 0.2360
Epoch 9: Train Loss = 0.3506, Eval Loss = 0.2363
Epoch 10: Train Loss = 0.3368, Eval Loss = 0.2359
Epoch 11: Train Loss = 0.3198, Eval Loss = 0.2360
Epoch 12: Train Loss = 0.3053, Eval Loss = 0.2360
Epoch 13: Train Loss = 0.2929, Eval Loss = 0.2361


In [ ]:
print("Best hyperparameters:", study.best_trial.params)


In [ ]:
# predict

mlp_model.eval()
with torch.no_grad():
    y_pred = mlp_model(torch.tensor(x_test_mlp, dtype=torch.float32).to(device))


y_pred = y_pred.cpu().numpy()
errors = y_pred - y_test_mlp  # assuming y_test is numpy array
mse = np.mean(errors**2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(errors))

print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")

